
# Archived

Train using entire dataset

In [ ]:
trainloader, testloader, outputs = get_dataloader(config["victim"]["data"])
model = get_model(config["attacker"], outputs)
result = training(model, trainloader, testloader, config["epochs"],
                           optimizer=torch.optim.Adam(model.parameters(), lr=config["learning_rate"]),
                           loss=torch.nn.CrossEntropyLoss())

Testing Time outputs

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    qtl = QueryVictim(trainloader, 1000, sampling='coreset_cross')
print(f'Query Time:{prof.key_averages().self_cpu_time_total/1e6}s)

Visualize random Images from CIFAR-10

In [ ]:
class_names = ['air_plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# get some random training images
plt.figure(figsize=(10, 10))
counter = 0
for images, labels in trainloader:
    for i, img in enumerate(images):
        plt.subplot(5, 5, counter+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        img = img/2 + 0.5   # unmornalize
        # convert (C,H,W) to (H,W,C)
        img_show = np.transpose(img, (1, 2, 0))
        plt.imshow(img_show, cmap=plt.cm.binary)
        plt.xlabel(class_names[int(labels[i])], color='red', fontsize='large')
        counter += 1
        if counter == 25:
            break
    if counter == 25:
        break

plt.show()

Load Victim from File

In [ ]:
victim = ResNet34(3, ResBlock, outputs=10)
victim.load_state_dict(torch.load('Victim_resnet34'))
victim.eval()

# Config Archived

import torch

from victim.__init__ import *

config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 80,
    "query_size":5000,
    "query_type": 'random',
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "victim": { "data": CIFAR_100,  "model_name": RESNET_56 },
    "attacker": RESNET50
}

parameters = {
        "query_size": [20000],
        "query_type": ['random','coreset', 'coreset_cross'],
        "victim":[{ "data": CIFAR_10, "model_name": RESNET50}],
        "attacker": [RESNET34]
}

# Parameter Runs

## Best Query Type, Best Query sizes on 2 datasets

### Run1 - CIFAR10 - A_RESNET34 - V_RESNET50

config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 100,
    "query_size":5000,
    "query_type": 'coreset',
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "victim": { "data": CIFAR_100,  "model_name": RESNET_56 }
}

parameters = {
        "query_size": [10000, 20000, 30000, 40000, 50000],
        "query_type": ['random', 'coreset', 'coreset_cross'],
        "victim":[{ "data": CIFAR_10,  "model_name": RESNET50}]
}

### Run2 - CIFAR10 - A_RESNET34 - V_RESNET50

config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 80,
    "query_size":10000,
    "query_type": 'coreset',
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "victim": { "data": CIFAR_10,  "model_name": RESNET50 },
}

parameters = {
        "query_size": [10000, 20000, 30000, 40000],
        "query_type": ['coreset', 'coreset_cross'],
        "victim":[{ "data": CIFAR_10, "model_name": RESNET50}]
}

### Run3 - CIFAR10 - A_RESNET34 - V_RESNET50

config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 80,
    "query_size":5000,
    "query_type": 'random',
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "victim": { "data": CIFAR_100,  "model_name": RESNET_56 },
    "attacker": RESNET50
}

parameters = {
        "query_size": [20000],
        "query_type": ['random','coreset', 'coreset_cross'],
        "victim":[{ "data": CIFAR_10, "model_name": RESNET50}],
        "attacker": [RESNET34]
}

### Run4 - A_RESNET34 - Random Query

config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 80,
    "query_size":10000,
    "query_type": 'random',
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "victim": {"data": CIFAR_10, "model_name": VGG19_BN},
    "attacker": RESNET34
}

parameters = {
        "query_size": [10000, 20000, 30000, 40000, 10000],
        "query_type": ['random'],
        "victim":[{ "data": CIFAR_10, "model_name": VGG19_BN}, 
                  { "data": CIFAR_100, "model_name": VGG19_BN}, 
                  {"data": CIFAR_100, "model_name": RESNET_56 }],
        "attacker":[RESNET34]
}

### Trial Run for coreset

config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 10,
    "query_size":10000,
    "query_type": 'random',
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "victim": {"data": CIFAR_10, "model_name": VGG19_BN},
    "attacker": RESNET34
}

parameters = {
        "query_size": [50000],
        "query_type": ['coreset','random'],
        "victim":[{ "data": CIFAR_10, "model_name": RESNET50}],
        "attacker":[RESNET34]
}

### Run5 - A_RESNET34 - Coreset Query

config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 80,
    "query_size":10000,
    "query_type": 'random',
    "victim": { "data": CIFAR_10,  "model_name": RESNET50 },
    "attacker": RESNET34,
    "klogits": 2
}

parameters = {
        "query_size": [10000, 20000, 30000, 40000, 50000],
        "query_type": ['coreset','coreset_cross'],
        "victim":[{ "data": CIFAR_10, "model_name": RESNET50},
                  { "data": CIFAR_10, "model_name": VGG19_BN}, 
                  { "data": CIFAR_100, "model_name": VGG19_BN}, 
                  {"data": CIFAR_100, "model_name": RESNET_56 }],
        "attacker":[RESNET34],
        "klogits": [0, 3, 10]
}




